In [81]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as spy
from tqdm import tqdm
import time
import pickle


# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('display.max_rows',200)
pd.set_option('display.max_colwidth',None)

from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix

# icd9 or icd10 (seprerately)

In [82]:
ICD_VER=10

## Check icd related to heart failure

In [83]:
d_icd=pd.read_table("/data/mimic-iv-0.4/hosp/d_icd_diagnoses.csv", sep=',', index_col=0)
d_icd.index=d_icd.index.str.strip()
d1=d_icd.loc[d_icd['long_title'].str.lower().str.contains('heart failure')]

## heart failure icd selection

In [84]:
heart_icd10=['I509','I110','I5020','I5021','I5022','I5023','I5030','I5031','I5032','I5033','I5040','I5041','I5042','I5043','I50810','I50811','I50812',
             'I50813','I50814','I5082','I5083','I5084']

heart_icd9=['40201','40211','40291','4281','4289','4280','42820','42821','42822','42823','42830',
            '42831','42832','42833','42840','42841','42842','42843']

d_icd.loc[heart_icd10+heart_icd9]

,icd_version,long_title
icd_code,,
I509,10,"Heart failure, unspecified"
I110,10,Hypertensive heart disease with heart failure
I5020,10,Unspecified systolic (congestive) heart failure
I5021,10,Acute systolic (congestive) heart failure
I5022,10,Chronic systolic (congestive) heart failure
I5023,10,Acute on chronic systolic (congestive) heart failure
I5030,10,Unspecified diastolic (congestive) heart failure
I5031,10,Acute diastolic (congestive) heart failure
I5032,10,Chronic diastolic (congestive) heart failure


# explore diagnoses file

## obtain patients using icd9 or icd10 seperately

In [85]:
diag_tmp=pd.read_table("/data/mimic-iv-0.4/hosp/diagnoses_icd.csv", sep=',', index_col=0).reset_index()
icd9_pat_tmp=set(diag_tmp.loc[(diag_tmp['icd_version']==9)]['subject_id'])
icd10_pat_tmp=set(diag_tmp.loc[(diag_tmp['icd_version']==10)]['subject_id'])
len(icd9_pat_tmp)
len(icd10_pat_tmp)
icd_both=icd9_pat_tmp&icd10_pat_tmp
len(icd_both)
icd9_pat_only=list(icd9_pat_tmp-icd_both)
icd10_pat_only=list(icd10_pat_tmp-icd_both)
len(icd9_pat_only)
len(icd10_pat_only)

139033

105584

23495

115538

82089

In [86]:
## 질병진단 테이블
diag = pd.read_table('/data/mimic-iv-0.4/hosp/diagnoses_icd.csv', sep=',')
diag.dropna(inplace=True)
diag['icd_code']=diag['icd_code'].str.strip()
diag=diag.loc[diag['icd_version']==ICD_VER]

if ICD_VER==9:
    diag=diag.loc[diag['subject_id'].isin(icd9_pat_only)]
    diag
if ICD_VER==10:
    diag=diag.loc[diag['subject_id'].isin(icd10_pat_only)]
    diag
diag_stat = diag['icd_code'].value_counts().to_frame().reset_index()
diag_stat.columns = ['icd_code', 'count']

## ICD to 진단명
d_icd = pd.read_table('/data/mimic-iv-0.4/hosp/d_icd_diagnoses.csv', sep=',')
d_icd['icd_code']=d_icd['icd_code'].str.strip()

## diag에 진단명 merge
diag_title = pd.merge(diag_stat, d_icd, on='icd_code', how='left')
diag_title.head(100)

,subject_id,hadm_id,seq_num,icd_code,icd_version
2202718,11582944,21283853,3,O99280,10
2202719,11582944,21283853,2,E039,10
2202720,11582944,21283853,4,O621,10
2202721,11582944,21283853,6,Z3A40,10
2202722,11582944,21283853,5,Z370,10
...,...,...,...,...,...
4677907,19118815,29566765,2,C329,10
4677908,19118815,29566765,4,Z13220,10
4677909,19118815,29566765,6,Z87891,10
4677910,19118815,29566765,3,G40909,10


,icd_code,count,icd_version,long_title
0,I10,32577,10.0,Essential (primary) hypertension
1,E785,27787,10.0,"Hyperlipidemia, unspecified"
2,Z23,24777,10.0,Encounter for immunization
3,Z87891,22400,10.0,Personal history of nicotine dependence
4,K219,17999,10.0,Gastro-esophageal reflux disease without esophagitis
5,F329,16344,10.0,"Major depressive disorder, single episode, unspecified"
6,Z3800,15007,10.0,"Single liveborn infant, delivered vaginally"
7,F419,13977,10.0,"Anxiety disorder, unspecified"
8,I2510,13049,10.0,Atherosclerotic heart disease of native coronary artery without angina pectoris
9,N179,11311,10.0,"Acute kidney failure, unspecified"


# code mapping
* * H: hypertension
* * F: heart failure
* * N: etc

In [87]:
if ICD_VER==10:
    hyper_icd=['I10']
    heart_icd=heart_icd10.copy()
    
if ICD_VER==9:
    hyper_icd=['4019'] #icd9
    heart_icd= heart_icd9.copy()
     
def code_mapper(x):
    if x in hyper_icd:
        return 'H'
    if x in heart_icd:
        return 'F'
    return 'N'


diag['HFN'] = diag['icd_code'].map(code_mapper)
diag['HFN'].value_counts()
diag

N    1215994
H      32577
F      16635
Name: HFN, dtype: int64

,subject_id,hadm_id,seq_num,icd_code,icd_version,HFN
2202718,11582944,21283853,3,O99280,10,N
2202719,11582944,21283853,2,E039,10,N
2202720,11582944,21283853,4,O621,10,N
2202721,11582944,21283853,6,Z3A40,10,N
2202722,11582944,21283853,5,Z370,10,N
...,...,...,...,...,...,...
4677907,19118815,29566765,2,C329,10,N
4677908,19118815,29566765,4,Z13220,10,N
4677909,19118815,29566765,6,Z87891,10,N
4677910,19118815,29566765,3,G40909,10,N


# Extract only hypertension patient data

In [88]:
all_hyper_pat=diag.loc[diag['HFN']=='H', 'subject_id'].unique()
len(all_hyper_pat)
diag_hyper=diag.loc[diag['subject_id'].isin(all_hyper_pat)]
diag_hyper=diag_hyper.drop(columns=['seq_num', 'icd_version'])
diag_hyper

21787

,subject_id,hadm_id,icd_code,HFN
2348969,15965986,28778976,I6521,N
2348970,15965986,28778976,Z3A39,N
2348971,15965986,28778976,R200,N
2348972,15965986,28778976,Z370,N
2348973,15965986,28778976,Z22330,N
...,...,...,...,...
4677906,19118815,29566765,I10,H
4677907,19118815,29566765,C329,N
4677908,19118815,29566765,Z13220,N
4677909,19118815,29566765,Z87891,N


# append diagnosis date

In [89]:
adms = pd.read_table('/data/mimic-iv-0.4/core/admissions.csv', sep=',')[['hadm_id','admittime']]
diag_hyper_adtime = pd.merge(diag_hyper, adms, how='left', on='hadm_id')
diag_hyper_adtime['admittime']=pd.to_datetime(diag_hyper_adtime['admittime'])
diag_hyper_adtime
diag_hyper_adtime.dtypes
diag_hyper_adtime['HFN'].value_counts()

,subject_id,hadm_id,icd_code,HFN,admittime
0,15965986,28778976,I6521,N,2117-06-08 19:45:00
1,15965986,28778976,Z3A39,N,2117-06-08 19:45:00
2,15965986,28778976,R200,N,2117-06-08 19:45:00
3,15965986,28778976,Z370,N,2117-06-08 19:45:00
4,15965986,28778976,Z22330,N,2117-06-08 19:45:00
...,...,...,...,...,...
547916,19118815,29566765,I10,H,2189-03-05 09:38:00
547917,19118815,29566765,C329,N,2189-03-05 09:38:00
547918,19118815,29566765,Z13220,N,2189-03-05 09:38:00
547919,19118815,29566765,Z87891,N,2189-03-05 09:38:00


subject_id             int64
hadm_id                int64
icd_code              object
HFN                   object
admittime     datetime64[ns]
dtype: object

N    510204
H     32577
F      5140
Name: HFN, dtype: int64

# Divide patients into three groups
* cl0) Only diagnosed with hypertension and never with heart failure
* cl1) Diagnosed with heart failure after being diagnosed with hypertension
* np.nan) Diagnosed with heart failure before hypertension

In [90]:
def classifying_subject(df):
    H_adtime=df.loc[df['HFN']=='H', 'admittime'].min()

    if 'F' in df['HFN'].value_counts().index:
        F_adtime=df.loc[df['HFN']=='F', 'admittime'].min()

        if H_adtime<F_adtime:
            return 'cl1_subj'
        else:
            return 'NAN'
    else:
        return 'cl0_subj'
    
subj_class_group=diag_hyper_adtime.groupby('subject_id')[['HFN','admittime']].apply(classifying_subject)

In [91]:
subj_class_group
subj_class_group.value_counts()

subject_id
10001401    cl0_subj
10002348    cl0_subj
10002430         NAN
10002443    cl0_subj
10002495         NAN
              ...   
19997843    cl0_subj
19999204    cl0_subj
19999466    cl0_subj
19999784    cl0_subj
19999828    cl0_subj
Length: 21787, dtype: object

cl0_subj    19909
NAN          1089
cl1_subj      789
dtype: int64

# generate class0, class1 data table
* Extract previously diagnosed ICD codes from the time when hypertension was first diagnosed
* Find the difference between when hypertension was first diagnosed and when heart failure was first diagnosed. (HF_diff)

In [92]:
def prev_icd_before_first_hypertension(df, cl):
    H_adtime=df.loc[df['HFN']=='H', 'admittime'].min()
    
    df_sorted=df.sort_values(by='admittime')
    df_min_adtime=df_sorted.drop_duplicates(subset='icd_code')
    df_prev_dis=df_min_adtime.loc[df_min_adtime['admittime']<=H_adtime]

    df_prev_dis['diff_day']=H_adtime-df_prev_dis['admittime']
    df_prev_dis['diff_year']=df_prev_dis['diff_day'].map(lambda x: int(x.days/365)+1)
    df_prev_dis=df_prev_dis.set_index('icd_code')['diff_year']

    if cl=='cl1':
        F_adtime=df.loc[df['HFN']=='F', 'admittime'].min()
        HF_diff_year=np.ceil((F_adtime-H_adtime).days/365)
    if cl=='cl0':
        HF_diff_year='Not_avail'
    
    res1=pd.Series({'prev_icds':df_prev_dis, 'HF_diff': HF_diff_year, 'H_time':H_adtime})
    return res1

def generate_data_of_specific_class(cl, subj_class_group, diag_hyper_adtime):
    if cl=='cl1':
        cl_subj=subj_class_group.loc[subj_class_group=='cl1_subj'].index
    if cl=='cl0':
        cl_subj=subj_class_group.loc[subj_class_group=='cl0_subj'].index
    cl_dat1=diag_hyper_adtime.loc[diag_hyper_adtime['subject_id'].isin(cl_subj)]
    
    # 1) 가장 처음 hypertension 진단받은 시점으로부터 이전 진단 받은 icd code 추출하기 (H진단 지점과 이전 질병 진단 시점의 차이도 함께 )
    # 2) 가장 처음 hypertension 진단받은 시점과 가장 처음 heart failure 진단받은 시점의 차이 구하기
    #   o cl1) 차이를 입력 
    #   o cl0) heart failure 진단 받은 적이 없으므로 np.nan을 입력
    
    cl_dat_table_tmp=cl_dat1.groupby('subject_id')[['icd_code','HFN','admittime']].apply(prev_icd_before_first_hypertension, cl)
    
    res_df=pd.DataFrame(columns=['subject_id','HF_diff', 'H_time','prev_icd','Hprev_diff'])

    int_ind=0
    for subj, row in cl_dat_table_tmp.iterrows():
        prev_dis, HF_diff, H_time=row
        for prev_icd in prev_dis.index:
            res_df.loc[int_ind]=[subj, HF_diff, H_time, prev_icd, prev_dis[prev_icd]]
            int_ind+=1
    
    res_df_final=res_df.pivot_table(index=['subject_id','HF_diff', 'H_time'], columns='prev_icd', values='Hprev_diff').fillna(0)
    
    return res_df_final

In [93]:
cl1_dat=generate_data_of_specific_class('cl1', subj_class_group, diag_hyper_adtime)
cl1_dat

cl0_dat=generate_data_of_specific_class('cl0', subj_class_group, diag_hyper_adtime)
cl0_dat

,,prev_icd,A045,A047,A0472,A0811,A0839,A084,A09,A401,A403,A408,...,Z9841,Z9842,Z9851,Z9861,Z9862,Z9884,Z9889,Z992,Z993,Z9981
subject_id,HF_diff,H_time,,,,,,,,,,,,,,,,,,,,,
10014354,2.0,2146-10-08 23:47:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10018328,1.0,2154-02-05 21:58:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10041894,3.0,2140-12-05 03:09:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10049041,1.0,2163-12-19 14:48:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10070201,1.0,2140-07-15 18:24:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19954460,1.0,2156-03-14 21:38:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19970101,3.0,2184-02-21 18:33:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19977875,1.0,2162-03-26 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,,prev_icd,A0100,A020,A021,A0222,A030,A039,A044,A045,A047,A0471,...,Z98818,Z9882,Z9884,Z9889,Z98890,Z9911,Z992,Z993,Z9981,Z9989
subject_id,HF_diff,H_time,,,,,,,,,,,,,,,,,,,,,
10001401,Not_avail,2131-06-04 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002348,Not_avail,2112-11-30 22:22:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002443,Not_avail,2183-10-17 23:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002807,Not_avail,2152-03-30 16:09:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003385,Not_avail,2131-09-26 02:48:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19997843,Not_avail,2120-11-17 23:16:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19999204,Not_avail,2146-05-30 16:43:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19999466,Not_avail,2116-08-30 15:21:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Add age & gender information for each subject

In [94]:
def add_subject_info(df):
    pats = pd.read_table('/data/mimic-iv-0.4/core/patients.csv',sep=',')
    df=df.reset_index()
    df.columns.name=None

    df=pd.merge(df, pats[['subject_id','gender','anchor_age','anchor_year']], on='subject_id', how='inner')
    df['H_age']=df['H_time'].dt.year-df['anchor_year']+df['anchor_age']
    df=df[['subject_id','gender','H_time','H_age']+list(df.columns[3:-4])+['HF_diff']]
    return df
    
cl1_dat=add_subject_info(cl1_dat)
cl1_dat
cl0_dat=add_subject_info(cl0_dat)
cl0_dat

with open('./results/data_cl1_icd{}_all.pickle'.format(ICD_VER), 'wb') as f:
    pickle.dump(cl1_dat, f)
    
with open('./results/data_cl0_icd{}_all.pickle'.format(ICD_VER), 'wb') as f:
    pickle.dump(cl0_dat, f)

,subject_id,gender,H_time,H_age,A045,A047,A0472,A0811,A0839,A084,...,Z9842,Z9851,Z9861,Z9862,Z9884,Z9889,Z992,Z993,Z9981,HF_diff
0,10014354,M,2146-10-08 23:47:00,60,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
1,10018328,F,2154-02-05 21:58:00,83,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,10041894,M,2140-12-05 03:09:00,78,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,10049041,M,2163-12-19 14:48:00,59,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,10070201,M,2140-07-15 18:24:00,74,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,19954460,F,2156-03-14 21:38:00,88,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
785,19970101,M,2184-02-21 18:33:00,87,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
786,19977875,F,2162-03-26 14:00:00,77,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
787,19983009,M,2142-09-04 17:52:00,56,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,subject_id,gender,H_time,H_age,A0100,A020,A021,A0222,A030,A039,...,Z9882,Z9884,Z9889,Z98890,Z9911,Z992,Z993,Z9981,Z9989,HF_diff
0,10001401,F,2131-06-04 00:00:00,89,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
1,10002348,F,2112-11-30 22:22:00,77,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
2,10002443,M,2183-10-17 23:20:00,53,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
3,10002807,M,2152-03-30 16:09:00,66,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
4,10003385,M,2131-09-26 02:48:00,41,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19904,19997843,M,2120-11-17 23:16:00,47,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
19905,19999204,M,2146-05-30 16:43:00,61,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
19906,19999466,M,2116-08-30 15:21:00,33,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
19907,19999784,M,2119-12-05 09:31:00,57,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
